In [1]:
import sys
sys.path.append("..")
from common_utils import set_data_home
set_data_home("/mnt/d")
from common_utils import DATA_HOME, join
from lstm.sales_data import Sales_Dataset, join

sd = Sales_Dataset(join(DATA_HOME, "sales_data"))
len(sd)


/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


54

In [2]:
# print(sd.O.describe())
# print(sd.TR.describe())
# print(sd.TS.describe())
# print(sd.S.describe())
# print(sd.TR[sd.TR.index == (1, "2014-01-01")])
sd[4]
# print(test)

/mnt/d/SOTA/lstm/../lstm/sales_data.py:144: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sale_data[sale_data.family == sale_data.family[0]].index


In [ ]:
%debug